In [63]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, FewShotPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser, PydanticOutputParser, SimpleJsonOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage, trim_messages
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

from langchain_chroma import Chroma

from langchain_community.chat_message_histories import ChatMessageHistory

from pydantic import BaseModel, Field, model_validator

from operator import itemgetter

In [2]:
prompt = ChatPromptTemplate.from_template(
    '주제 {topic}에 대해 짧은 설명을 해주세요.'
)
output_parser = StrOutputParser()
model = ChatOpenAI(model='gpt-5-nano')

In [3]:
chain = (
    {'topic':RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

print(chain.invoke('더블딥'))

더블딥은 분야에 따라 다른 의미로 쓰입니다. 두 가지 흔한 해석을 간단히 소개합니다.

- Double DQN(강화학습): Q-학습의 과대추정을 줄이기 위해 정책 선택과 가치 평가를 서로 다른 두 네트워크로 분리하는 기법입니다. 이렇게 하면 에이전트의 학습이 더 안정적으로 이뤄집니다.
- 이중침체(경제학): 경기가 한 차례 회복을 보이다가 다시 침체에 빠지는 현상을 말합니다. 보통 "이중 침체"로도 불립니다.

원하시는 분야를 알려주시면 그에 맞춰 더 간단히 설명해 드릴게요.


In [4]:
prompt = ChatPromptTemplate.from_template(
    '주제 {topic}에 대해 짧은 설명을 해주세요.'
)
output_parser = StrOutputParser()
model = ChatOpenAI(model='gpt-5-nano', 
                   #temperature=1,
                   #max_completion_tokens=1000
                   )

In [5]:
chain = (
    {'topic':RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

for chunk in chain.stream('더블딥'):
    print(chunk, end='', flush=True)

혹시 강화학습의 용어인 "더블 DQN(Double DQN)"를 말씀하시는 건가요? 아니면 다른 뜻의 "더블딥"을 의미하시는지 확인이 필요합니다. 만약 Double DQN이라면 짧은 설명은 다음과 같습니다:

- 더블 DQN은 Q-값의 과대추정을 줄이기 위해 정책 선택과 가치 평가를 서로 다른 두 네트워크로 분리해 학습합니다. 하나의 네트워크가 행동을 선택하고, 다른 네트워크가 그 선택의 가치를 평가해 타깃 값을 업데이트합니다.

다른 뜻이라면 어떤 주제를 원하시는지 알려주시면 짧게 설명해 드리겠습니다.

In [6]:
prompt = ChatPromptTemplate.from_template(
    '주제 {topic}에 대해 짧은 설명을 해주세요.'
)
output_parser = StrOutputParser()
model = ChatOpenAI(model='gpt-5-nano', 
                   #temperature=1,
                   #max_completion_tokens=1000
                   )

In [7]:
chain = (
    {'topic':RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

print(chain.batch([{'topic':'더블딥'}, {'topic':'인플레이션'}])[0])
print(chain.batch([{'topic':'더블딥'}, {'topic':'인플레이션'}])[1])

더블딥은 한 번 침체를 벗어나려는 듯 보이다가 다시 경기 하강에 빠지는 현상을 말합니다. 이중 침체라고도 부릅니다.

- 보통 실질 GDP가 두 분기 연속 감소하거나, 짧은 회복 뒤에 다시 약화되며 재침체가 나타납니다.
- 원인으로 수요 부진, 금융 여건 악화, 정책 반응의 지연 등이 작용하고, 고용과 투자에도 큰 타격이 있습니다.
다음은 인플레이션에 대한 짧은 설명입니다.

- 정의: 전반적인 물가가 상승해 같은 돈으로 살 수 있는 상품·서비스의 양이 줄어드는 현상.
- 측정: 소비자물가상승률(CPI) 등으로 연간 물가 상승률을 나타냄.
- 주된 원인: 수요 증가에 따른 수요주도 인플레이션, 생산비 상승·원자재 가격 상승 등 비용주도 인플레이션, 물가 기대의 상승 등.
- 영향: 실질 구매력 감소, 저축 가치 변화, 금리와 고용에 영향.
- 정책: 중앙은행의 물가안정 목표와 금리 정책으로 수요를 관리하려는 노력.


In [8]:
analysis_prompt = ChatPromptTemplate.from_template('이 대답을 영어로 번역해주세요: {answer}')
composed_chain = (
    {'answer':chain}
    | analysis_prompt
    | model
    | output_parser
)
print(composed_chain.invoke({'topic':'더블딥'}))

A Double Dip is the phenomenon where the economy, after experiencing one recession, briefly recovers and then falls back into recession. The recovery does not rise above the previous trough, which is a characteristic of the pattern, and it is often described as a "double bottom" pattern. Causes include various factors such as policy lags, weak demand, and deteriorating financial conditions.


In [9]:
prompt = ChatPromptTemplate.from_template('{topic}에 대해서 짧은 설명을 해주세요.')
chain = (
    {'topic':RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)
analysis_prompt = ChatPromptTemplate.from_template('이 대답을 영어로 번역해주세요.:{answer}')
composed_chain_with_lambda = (
    chain
    | (lambda input: {'answer':input})
    | analysis_prompt
    | model
    | output_parser
)
print(composed_chain_with_lambda.invoke({'topic':'더블딥'}))

A double dip refers to an economic phenomenon in which the economy experiences a period of recovery, only to slip back into a recession. In other words, after a rebound GDP falls again and employment and other indicators deteriorate, a pattern sometimes likened to a “W-shaped recovery.” The main causes include weak demand, the aftereffects of financial shocks, and lags in the effects of policy.


In [10]:
composed_chain_with_pipe = chain.pipe(
    lambda input: {'answer':input}, analysis_prompt, model, output_parser
)
print(composed_chain_with_pipe.invoke({'topic':'더블딥'}))

Double dip is primarily an economic term that refers to a phenomenon where, after one recession ends, there is a brief recovery before slipping back into recession. In other words, it describes a situation in which two recessions occur in succession. If you’d like, I can also explain its meaning in other fields.


In [11]:
kor_chain = (
    ChatPromptTemplate.from_template('{topic}에 대한 짧은 설명을 해주세요.')
    | model
    | output_parser
)
eng_chain = (
    ChatPromptTemplate.from_template('{topic}에 대해 짧게 영어로 설명을 해주세요.')
    | model
    | output_parser
)
parallel_chain = RunnableParallel(kor=kor_chain, eng=eng_chain)
result = parallel_chain.invoke({'topic':'더블딥'})
print(result['kor'])
print(result['eng'])

더블딥(DQN)의 한 버전으로, Q-학습의 과대추정을 줄이기 위해 학습에서 행동 선택과 평가를 서로 다른 네트워크가 담당하도록 하는 방법입니다. 
구현 방식은: 온라인 네트워크가 s'에서 최적 행동 a'를 선택하고, 타깃 네트워크가 그 행동의 값을 평가해 y = r + γ Q(s', a'; θ^-) 를 목표로 학습합니다. 타깃 네트워크는 주기적으로 또는 소프트 업데이트로 가중치를 반영합니다. 
이렇게 두 네트워크를 분리하면 학습의 안정성이 높아지고 성능이 향상되는 경향이 있습니다.
Double DQN is an extension of DQN that reduces overestimation in Q-learning.

- It uses two neural networks: an online network for selecting actions and a separate target network for evaluating them.
- During update, actions for the next state are chosen by the online network, but their values are estimated by the target network.
- The target value is y = r + gamma * Q_target(next_state, argmax_a Q_online(next_state, a)).
- The target network is updated slowly to stabilize learning.

This decoupling makes learning more stable and often improves performance.


In [12]:
prompt_template = ChatPromptTemplate.from_messages(
    [('system', '당신은 유능한 금융 조언가입니다.'),
     ('user', '주제 {topic}에 대하여 금융 관련 조언을 해주세요.')]
)
chain = (
    {'topic':RunnablePassthrough()}
    | prompt_template
    | model
    | output_parser
)
print(chain.invoke({'topic':'더블딥'}))

주제: 더블딥(이중 침체)에 대한 금융 관련 조언

더블딥은 경제가 일시적으로 회복 국면에 접어든 뒤 다시 침체로 전환되는 시나리오를 말합니다. 원인으로는 성장 모멘텀의 약화, 실물 수요의 급감, 고물가나 금리 인상으로 인한 차입 비용 증가, 정책 효과의 한계 등이 있습니다.

주요 신호(모니터링 포인트)
- GDP 성장률의 재하강 또는 기대치 하회
- 고용 지표의 악화 또는 고용회복의 이질성
- 제조업/서비스업 PMI의 재하강
- 소비심리/소비지출 부진
- 금리 인상 속도 조절 여부 또는 금융시장 불안
- 가계부채 부담 증가와 재정/통화 정책의 변화

더블딥에 대비한 투자 원칙
- 위험 관리 우선: 원금을 지키는 것이 우선이 될 수 있습니다. 과도한 위험 자산 비중은 recession 시 손실을 키울 수 있습니다.
- 분산과 다변화: 국내외 주식, 채권, 현금성 자산, 금 등 대체자산까지 폭넓은 분산이 필요합니다.
- 질 좋은 자산 위주: 안정적 현금흐름과 재무구조가 탄탄한 기업의 주식, 등급이 높은 채권 중심으로 구성하는 것이 방어적입니다.
- 현금흐름 관리: 비상금 확보와 단기 필요자금의 원천을 구분해 운용합니다.
- 긴 호흡의 자산배분: 단발성 타이밍보다는 시나리오 기반의 재조정이 필요합니다.

실전 전략(일반적인 가이드)
1) 유동성 확보
- 비상금으로 6-12개월의 생활비에 해당하는 자금을 현금성 자산으로 보유.
- 단기 국채/머니마켓펀드처럼 쉽게 현금화 가능한 자산 비중을 늘려 변동성 대응력을 높이세요.

2) 방어적 주식과 질 좋은 고배당/필수소비재 비중 확대
- 헬스케어, 필수소비재, 유틸리티 등 경기 민감도 낮은 섹터의 비중을 약간 확대.
- 변동성이 큰 성장주보단 품질주 중심으로 포트폴리오를 안정화합니다.

3) 질 높은 채권 중심의 고정수익 강화
- 단기에서 중기 만기 투자등급 채권, 국채 중심으로 구성해 이자소득과 원금보전을 강화.
- 금리하락 국면에 대비해 듀레이션 관리도 병행(필요 시 적정한 듀레이션 조정).

4) 인플레이션과 금리

In [13]:
prompt_template = ChatPromptTemplate.from_messages(
    [('system', '당신은 유능한 금융 조언가입니다.'),
     ('user', '주제 {topic}에 대하여 금융 관련 조언을 해주세요.')]
)
chain = (
    #{'topic':RunnablePassthrough()}
    prompt_template
    | model
    | output_parser
)
print(chain.invoke({'topic':'더블딥'}))

다음은 주제인 “더블딥”(이중 침체)에 대해 일반적으로 참고할 수 있는 금융 관련 조언입니다. 개인 상황에 맞춘 구체적 투자 조언은 신중히 전문가와 상의하시길 권합니다.

1) 더블딛이란 무엇인가
- 정의: 경제가 한 차례 심각한 침체를 겪은 뒤 짧은 반등을 보이다가 다시 침체로 내려앉는 상황을 말합니다. 흔히 V자 회복이 아닌 이중 하강 형태로 나타납니다.
- 원인 요인: 수요 부진의 재발, 고부채 부담, 공급 충격의 재현, 정책의 미스매치 등.
- 금융시장에의 영향: 불확실성과 변동성 증가, 위험자산의 하락 재개 가능성, 채권과 현금의 상대적 가치 변화.

2) 더블딥 시기에 주의해야 할 지표/징후
- 실물경제: 2분기 이상 GDP가 연속 감소하거나 고용이 급격히 악화될 때.
- 금융시장: 주가의 추가 급락 우려, 기업 실적 부진, 신용여건 악화.
- 정책환경: 중앙은행의 금리 방향성 전환 신호, 재정정책의 규모와 시기가 불확실해지는 경우.

3) 더블딥 가능성에 대비하는 일반적 전략
- 현금 흐름과 재무 건전성 강화
  - 비상금 마련: 생활비와 고정지출을 최소 6~12개월치 수준으로 확보하는 것이 일반적으로 권장됩니다.
  - 고금리 부채 우선 상환: 변동금리 대출이나 신용카드 부채 등 비용이 큰 부채를 먼저 정리합니다.
- 포트폴리오 방어력 강화
  - 자산배분의 기본에 충실: 주식과 채권의 균형, 위험 허용도에 맞춘 분산.
  - 방어적 주식 선호: 품질이 좋고 현금 흐름이 탄탄한 기업의 주식 비중 유지 또는 확대(필수소비재, 헬스케어, 유틸리티 등).
  - 고품질 채권 위주: 신용등급이 높은 단기/중기 채권으로 구성하고, 필요시 인플레이션 연동채(TIPS 등)도 고려.
- 시장 변동성 대비 다각화
  - 지역 다각화: 국내 시장에 의존하기보다 선진국/신흥국의 비중 균형 조정.
  - 자산 클래스 다각화: 주식 외에 현금성 자산, 채권, 일부 대체자산(헤지펀드 스타일의 저위험 전략 등)을 혼합.
- 투자 접근 방식
  - 달러 비용 평균화(Doll

In [14]:
prompt_template = ChatPromptTemplate.from_messages(
    [('system', '당신은 유능한 금융 조언가입니다.'),
     MessagesPlaceholder('msgs')]
)
chain = (
    #{'topic':RunnablePassthrough()}
    prompt_template
    | model
    | output_parser
)
print(chain.invoke({'msgs':[HumanMessage(content='안녕하세요. 더블딥에 대해 알려주세요.')]}))

반갑습니다. 더블딥은 주로 거시경제 용어로 쓰이며, “두 번째 경기 침체가 짧은 기간의 반등 뒤에 다시 찾아오는 현상”을 말합니다. 자세히 정리해 드릴게요.

1) 더블딥의 정의
- 기본 아이디어: 한 번의 경기 침체가 끝나고 회복 국면이 시작된 듯 보였는데, 그 회복이 이어지지 못하고 다시 경제가 위축되는 경우를 말합니다.
- 공식 판정과의 차이: 각 나라의 경기 사이클 판단기관은 다르게 정의합니다. 예를 들어 미국은 NBER가 경기 침체 여부를 판단하고, 두 번의 연속 마이너스 성장만으로 자동으로 규정하지 않는 경우가 많습니다. 따라서 “더블딥”은 주로 언론이나 시장 참가자 사이의 용어로도 쓰이고, 실질로는 두 번의 공식 침체 여부가 나와야 확정되기도 합니다.

2) 왜 생길 수 있나요? (주요 원인)
- 가계 부채 부담과 소비 약화로 인한 수요 급감
- 기업 투자 감소와 생산 차질로 인한 생산성 악화
- 정책 불확실성(금리 정책, 재정정책 방향의 변화)으로 인한 지연적 소비/투자
- 외부 충격(전쟁, 원자재 가격 급등락, 팬데믹 이후 회복의 바람직한 속도 저하)
- 금융시장 충격과 신용 여건 악화로 인한 재투자 위축

3) 흔히 보이는 징후/지표
- 분기 GDP가 연속해서 마이너스인 기간이 짧게 끝난 뒤 다시 마이너스로 돌아감(공식 판단과 시점 차이 가능)
- 실업률 상승 재개, 소비지표 약세(소매판매, 소비심리 악화)
- 제조업 PMI나 산업생산 지표의 재하락
- 인플레이션/금리 흐름이 정책을 어렵게 하거나, 정책 변화에 따른 기대 변화가 불확실해짐
- 금융시장 변동성 증가와 신용 여건 악화

4) 역사적 맥락
- 더블딥은 비교적 드물고, 특정 지역에서의 경기 사이클 맥락에서 언급되는 경우가 많습니다. 예를 들어 global 금융위기 이후 회복 국면에서 “다시 침체로 돌아설 수 있다”는 우려가 제기되기도 했지만, 공식적으로 두 번의 연속 마이너스 GDP로 확정되지는 않기도 했습니다. 따라서 상황마다 다르고, 데이터가 확정적으로 나오기 전까지는 단정하기 

In [15]:
prompt_template = ChatPromptTemplate.from_messages(
    [('system', '당신은 유능한 금융 조언가입니다.'),
     ('placeholder', '{msgs}')]
)
chain = (
    #{'topic':RunnablePassthrough()}
    prompt_template
    | model
    | output_parser
)
print(chain.invoke({'msgs':[HumanMessage(content='안녕하세요. 더블딥에 대해 알려주세요.')]}))

안녕하세요! 더블딥은 보통 경제가 한 차례 침체를 겪은 뒤 잠시 회복했다가 다시 침체로 들어서는 현상을 말합니다. 간단하게 정리해 드리면:

1) 정의
- 더블딥은 두 번의 경기 침체가 연이어 발생하는 상황을 가리킵니다.
- 일반적인 간이 기준으로는 실질 GDP가 두 분기 연속 감소하는 구간이 서로 떨어져 있다가 다시 침체로 이어지는 경우를 말하곤 합니다. 다만 공식 침체 여부는 각 나라의 경기연구기관(NBER 등)에서 판단합니다.

2) 왜 생길까(발생 원인)
- 수요 급감: 소비와 기업 설비 투자 등이 급락합니다.
- 금융 제약: 신용 경색, 이자율 급등 혹은 시장 불안이 지속될 때 회복이 흔들립니다.
- 정책 효과의 이중성: 경기부양책의 효과가 한동안만 지속되고, 이후 추가 지원이 부족하면 재침체로 이어질 수 있습니다.
- 외부 shock: 에너지 가격 급등/급락, 글로벌 공급망 붕괴 등 외부 충격이 재차 발생하면 회복이 흔들립니다.

3) 신호와 지표
- GDP 성장률이 두 분기 연속 마이너스로 접하면 분위기가 악화되기 시작합니다.
- 실업률 상승, 소비지출 약화, 제조업/서비스업 PMI 악화 등의 선행 지표가 약세를 보입니다.
- 금융시장 변수로는 주가 하락과 함께 채권 수익률 곡선의 변화, 신용스프레드 확대 등이 동반될 수 있습니다.
- 공식적으로는 각 국의 경기연구기관이 침체의 시작과 종료를 판단합니다.

4) 역사적 사례(참고)
- 1980년대 미국은 두 차례의 침체가 짧은 회복을 사이에 두고 이어진 사례가 언급되곤 합니다(더블딥에 대한 대표적 예로 자주 인용).
- 2008년 금융위기 이후 글로벌 회복 과정에서 더블딥 우려가 제기되었지만, 공식적으로 두 차례의 침체가 확정되지는 않는 경우가 많았습니다. 즉, “더블딥 우려”와 실제 침체 여부는 다를 수 있습니다.

5) 투자자 관점에서의 대응 포인트
- 방어적 주식 및 채권 구성: 경기 둔화 가능성에 대비해 헬스케어/필수소비재 등 방어적 섹터의 비중을 고려하고, 고품질의 단기/중기 채권으로 위험

In [16]:
examples = [
    {
        "question": "주식 투자와 예금 중 어느 것이 더 수익률이 높은가?",
        "answer": """
후속 질문이 필요한가요: 네.
후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
중간 답변: 주식 투자의 평균 수익률은 연 7%입니다.
후속 질문: 예금의 평균 이자율은 얼마인가요?
중간 답변: 예금의 평균 이자율은 연 1%입니다.
따라서 최종 답변은: 주식 투자
""",
    } ,
    {
        "question": "부동산과 채권 중 어느 것이 더 안정적인 투자처인가?",
        "answer": """
후속 질문이 필요한가요: 네.
후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
중간 답변: 부동산 투자의 위험도는 중간 수준입니다.
후속 질문: 채권의 위험도는 어느 정도인가요?
중간 답변: 채권의 위험도는 낮은 편입니다.
따라서 최종 답변은: 채권
""",
    },
]

example_prompt = PromptTemplate.from_template('질문: {question}\n답변: {answer}')
print(example_prompt.invoke(examples[0]).to_string())

질문: 주식 투자와 예금 중 어느 것이 더 수익률이 높은가?
답변: 
후속 질문이 필요한가요: 네.
후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
중간 답변: 주식 투자의 평균 수익률은 연 7%입니다.
후속 질문: 예금의 평균 이자율은 얼마인가요?
중간 답변: 예금의 평균 이자율은 연 1%입니다.
따라서 최종 답변은: 주식 투자



In [17]:
prompt = FewShotPromptTemplate(examples=examples, example_prompt=example_prompt, suffix='질문: {input}', input_variables=['input'])
print(prompt.invoke({'input':'부동산 투자의 장점은 무엇인가?'}).to_string())

질문: 주식 투자와 예금 중 어느 것이 더 수익률이 높은가?
답변: 
후속 질문이 필요한가요: 네.
후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
중간 답변: 주식 투자의 평균 수익률은 연 7%입니다.
후속 질문: 예금의 평균 이자율은 얼마인가요?
중간 답변: 예금의 평균 이자율은 연 1%입니다.
따라서 최종 답변은: 주식 투자


질문: 부동산과 채권 중 어느 것이 더 안정적인 투자처인가?
답변: 
후속 질문이 필요한가요: 네.
후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
중간 답변: 부동산 투자의 위험도는 중간 수준입니다.
후속 질문: 채권의 위험도는 어느 정도인가요?
중간 답변: 채권의 위험도는 낮은 편입니다.
따라서 최종 답변은: 채권


질문: 부동산 투자의 장점은 무엇인가?


In [18]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    Chroma,
    k=1
)

In [19]:
question = '부동산 투자의 장점은 무엇인가?'
print(f'입력질문: {question}')
selected_examples = example_selector.select_examples({'question':question})
for example in selected_examples:
    print('\n')
    print('# 입력과 가장 유사한 예제:')
    for k, v in reversed(example.items()):
        print(f'{k}: {v}')

입력질문: 부동산 투자의 장점은 무엇인가?


# 입력과 가장 유사한 예제:
answer: 
후속 질문이 필요한가요: 네.
후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
중간 답변: 부동산 투자의 위험도는 중간 수준입니다.
후속 질문: 채권의 위험도는 어느 정도인가요?
중간 답변: 채권의 위험도는 낮은 편입니다.
따라서 최종 답변은: 채권

question: 부동산과 채권 중 어느 것이 더 안정적인 투자처인가?


In [20]:
example_prompt = PromptTemplate(
    input_variables=['question', 'answer'],
    template='질문: {question}\n답변: {answer}'
)

prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix='다음은 금융 관련 질문과 답변의 예입니다.',
    suffix='질문: {question}\n답변:',
    input_variables=['input']
)

In [21]:
model = ChatOpenAI(model='gpt-5-nano')
chain = (
    prompt
    | model
    #| output_parser
)
response = chain.invoke({'question': '부동산 투자의 장점은 무엇인가?'})
print(response.content)

답변:
- 임대 수입을 통한 정기적 현금흐름: 임대를 통한 꾸준한 수입으로 현금 흐름이 안정될 수 있습니다.
- 자본 상승 가능성: 부동산 가치가 상승하면 매매 시 시세 차익을 얻을 수 있습니다.
- 레버리지 활용: 대출을 이용해 초기 자본 대비 더 큰 규모의 투자가 가능해 수익률을 확대할 수 있습니다.
- 인플레이션 헤지: 물가 상승 시 임대료와 자산가치가 함께 상승하는 경향이 있어 인플레이션에 상대적으로 강한 편입니다.
- 포트폴리오 다변화 및 위험 분산: 주식·채권과의 상관관계가 낮아 포트폴리오의 전반적 위험을 분산하는 데 도움이 됩니다.
- 세제 혜택 및 감가상각: 지역에 따라 세제 혜택이나 감가상각 혜택을 활용해 순이익이 개선될 수 있습니다.
- 관리 및 가치 창출 가능성: 리모델링이나 관리 개선을 통해 자산 가치를 직접 높일 수 있습니다.
- 실물 자산의 제어권: 물리적 자산을 보유하고 직접 관리하는 선택이 가능합니다.

참고: 지역·시장 상황에 따라 장점의 크기와 실현 가능성은 달라지며, 유동성은 일반적으로 채권/주식에 비해 낮은 편입니다.


In [22]:
parser = JsonOutputParser()
instructions = parser.get_format_instructions()
print(instructions)

Return a JSON object.


In [23]:
ai_response = '{"이름":"김철수", "나이":30}'
parsed_response = parser.parse(ai_response)
print(parsed_response)

{'이름': '김철수', '나이': 30}


In [25]:
model = ChatOpenAI(
    model='gpt-5-nano',
    temperature=0.0
)

In [35]:
class FinancialAdvice(BaseModel):
    setup: str = Field(description='금융 조언 상황을 설정하기 위한 질문. 이 질문은 반드시 물음표(?)로 끝나야 합니다.')
    advice: str = Field(description='질문을 해결하기 위한 금융 답변')

    @model_validator(mode='before')
    @classmethod
    def question_ends_with_question_mark(cls, values:dict) -> dict:
        setup = values.get('setup', '')
        if not setup.endswith('?'):
            raise ValueError('잘못된 질문 형식입니다. 질문은 ?로 끝나야 합니다.')
        return values

In [36]:
parser = PydanticOutputParser(pydantic_object=FinancialAdvice)
prompt = PromptTemplate(
    template='다음 금융 관련 질문에 답변해주세요.\n{format_instructions}\n질문:{query}\n',
    input_variables=['query'],
    partial_variables={'format_instructions':parser.get_format_instructions()}
)

chain = (
    prompt
    | model
    | parser
)

In [37]:
try:
    result = chain.invoke({'query':'부동산에 관련하여 금융 조언을 받을 수 있게 질문하라.'})
    print(result)
except Exception as e:
    print(f'오류 발생: {e}')

setup='전재산 6억원 규모의 부동산 매물에 대해 자금 조달 구조를 어떻게 설계하고 어떤 대출 옵션을 비교해야 하는지 구체적으로 조언해줄 수 있나요?' advice='투자 목적의 부동산에 대한 자금 조달은 대출 한도(LTV)와 금리 변화에 크게 좌우되므로 목표 수익과 보유 기간을 먼저 정의하는 것이 중요합니다. 아래 프레임을 따라 실행해 보세요. 1) 목표 설정: 예상 임대 수입, 허용 가능한 공실률, holding 기간, 목표 총수익률을 정합니다. 2) 다운페이먼트와 대출 구성: 보통 20~40%를 다운하고 나머지를 대출하는 방식이 일반적이나 은행별 요건에 따라 다릅니다. 투자용 대출의 LTV는 주거용 대출과 다를 수 있으며 금리는 고정형과 변동형 중 선택합니다. 3) 대출 옵션 비교 포인트: 투자용 부동산 담보대출, 상업용 대출, 프로젝트 파이낸싱 등의 조건(금리, 만기, 상환 일정, 수수료, 조기상환 페널티)을 은행별로 비교합니다. 4) 현금 흐름 모델링: 예상 임대료에서 관리비, 재산세, 보험료, 공실손실, 대출 원리금 상환액을 차감해 순현금흐름을 산출합니다. 예를 들어 가격 6억원, 다운 1.8억원(30%), 대출 4.2억원, 금리 4% 고정 20년의 경우 월 상환 약 2,600,000원 수준이고, 임대료를 3,200,000원, 관리비를 300,000원으로 가정하면 월 순현금흐름은 약 300,000원 수준이 될 수 있습니다. 이는 지역과 조건에 따라 달라지므로 여러 시나리오로 비교해야 합니다. 5) 리스크 관리: 금리 상승 시나리오와 공실 리스크를 반영해 비상금을 충분히 확보하고 지나친 레버리지를 피합니다. 6) 세금 및 규제 고려: 양도소득세, 재산세, 종합부동산세 등 투자부동산 관련 세금 영향을 미리 파악하고 세무 전문가와 상담합니다. 7) 실행 계획: 3곳 이상의 금융기관에서 금리 견적을 받고 각 시나리오의 현금 흐름을 비교한 뒤 최적의 구조를 선택합니다. 이 조언은 일반적인 가이드이며 개인 상황에 따라 다를 수 있으므로 실제 결정은 금융 전

In [40]:
json_prompt = PromptTemplate.from_template(
    '다음 질문에 대한 답변이 포함된 json 객체를 반환하세요.:{query}'
)
json_parser = SimpleJsonOutputParser()
json_chain = (
    json_prompt
    | model
    | json_parser
)
list(json_chain.stream({'query':'비트코인에 대한 짧은 한 문장 설명.'}))

[{},
 {'answer': ''},
 {'answer': '비'},
 {'answer': '비트'},
 {'answer': '비트코'},
 {'answer': '비트코인은'},
 {'answer': '비트코인은 중앙'},
 {'answer': '비트코인은 중앙 기관'},
 {'answer': '비트코인은 중앙 기관 없이'},
 {'answer': '비트코인은 중앙 기관 없이 P'},
 {'answer': '비트코인은 중앙 기관 없이 P2'},
 {'answer': '비트코인은 중앙 기관 없이 P2P'},
 {'answer': '비트코인은 중앙 기관 없이 P2P 네'},
 {'answer': '비트코인은 중앙 기관 없이 P2P 네트'},
 {'answer': '비트코인은 중앙 기관 없이 P2P 네트워크'},
 {'answer': '비트코인은 중앙 기관 없이 P2P 네트워크에서'},
 {'answer': '비트코인은 중앙 기관 없이 P2P 네트워크에서 거래'},
 {'answer': '비트코인은 중앙 기관 없이 P2P 네트워크에서 거래되고'},
 {'answer': '비트코인은 중앙 기관 없이 P2P 네트워크에서 거래되고 확인'},
 {'answer': '비트코인은 중앙 기관 없이 P2P 네트워크에서 거래되고 확인되는'},
 {'answer': '비트코인은 중앙 기관 없이 P2P 네트워크에서 거래되고 확인되는 탈'},
 {'answer': '비트코인은 중앙 기관 없이 P2P 네트워크에서 거래되고 확인되는 탈중'},
 {'answer': '비트코인은 중앙 기관 없이 P2P 네트워크에서 거래되고 확인되는 탈중앙'},
 {'answer': '비트코인은 중앙 기관 없이 P2P 네트워크에서 거래되고 확인되는 탈중앙화'},
 {'answer': '비트코인은 중앙 기관 없이 P2P 네트워크에서 거래되고 확인되는 탈중앙화된'},
 {'answer': '비트코인은 중앙 기관 없이 P2P 네트워크에서 거래되고 확인되는 탈중앙화된 암'},
 {'answer': '비트코인은 중앙 

In [43]:
parser = JsonOutputParser(pydantic_object=FinancialAdvice)
chain = (
    prompt
    | model
    | parser
)
chain.invoke({'query':'부동산 관련하여 금융 조언을 받을 수 있게 질문하라.'})

{'setup': '주거용 또는 투자용 부동산 구매를 고려 중인데, 제 소득·자산·신용 상태에 맞춘 최적의 자금 조달 전략과 대출 옵션을 어떻게 구성하면 좋을지 묻고 싶습니다?',
 'advice': '다음 순서로 진행해 보세요.\n- 예산과 목표를 명확히 설정: 주거용인지 투자용인지, 지역 선호, 예상 월 지출 한도, 예상 매매 시점\n- 신용도와 자산 점검: 신용점수 확인·채무 상태 정리·필요시 개선 계획 수립\n- 다운페이먼트와 대출 옵션 비교: 일반적으로 다운페이먼트 20% 이상 권장; 금리(고정/변동), 상환기간(10–30년), LTV/DTI 규정에 따라 달라짐\n- 총비용 산정: 매매가 외에 취득세, 등기비용, 중개 수수료, 보험료, 관리비, 재산세 등을 모두 계산\n- 대출 비교와 시나리오 분석: 월 상환액, 총이자, 만기 시 총지출을 여러 대출 옵션으로 비교\n- 투자 관점의 경우 현금 흐름 평가: 임대료 수입, 공실률, 운영비, 세제혜택(감가상각), 순현금 흐름과 수익률(CAP rate, IRR) 추정\n- 리스크 관리: 금리 상승, 실수요자의 자금 여력 악화에 대비한 긴급자금 확보(보통 3–6개월 지출분)\n- 실행을 위한 서류 준비: 신분증, 재직증명/소득증빙, 최근 금융거래내역, 자산 증빙, 대출상환내역\n- 실행 로드맵: 사전심사(pre-approval) → 매물 탐색 및 비교 → 계약 및 대출실행 순으로 진행\n원하시면 현재 소득 수준, 자산 구성, 신용 상태, 지역 선호, 희망 매매가대와 다운페이먼트 가능 금액 등을 알려 주세요. 그 정보를 바탕으로 더 구체적으로 맞춤 조언을 드리겠습니다.'}

In [45]:
prompt = ChatPromptTemplate.from_messages(
    [('system', '당신은 금융 상담사입니다. 사용자에게 최선의 금융 조언을 제공합니다.'),
     ('placeholder', '{msgs}')]
)

In [46]:
chat_history = ChatMessageHistory()
chat_history.add_user_message('저축을 늘리기 위해서 무엇을 할 수 있나요?')
chat_history.add_ai_message('저축 목표를 설정하고, 매달 자동 이체로 일정 금액을 저축하세요.')
chat_history.add_user_message('방금 뭐라고 했나요?')

In [48]:
chain = (
    prompt
    | model
    | StrOutputParser()
)
ai_response = chain.invoke({'msgs':chat_history.messages})
print(ai_response)

다시 말하면 이 두 가지를 제안한 거예요:
- 저축 목표를 세우기
- 매달 같은 금액을 자동으로 저축 계좌로 이체하도록 설정하기

구체적으로 실행 방법:
- 목표 설정: SMART 원칙으로 구체적이고 측정 가능하게 정합니다. 예: 6개월 안에 비상금 300만 원 확보, 1년 뒤 차량 수리비 150만 원 모으기 등.
- 예산 파악: 월소득에서 필수 지출과 불필요 지출을 파악하고, 절감 가능한 부분을 찾습니다.
- 자동이체 설정: 급여일이나 매월 정해진 날짜에 같은 금액이 자동으로 이체되도록 은행의 정기이체나 자동저축 서비스를 설정합니다. 시작은 여유 자금의 10~20% 정도로 시작해 점차 키우는 방식이 좋습니다.
- 계좌 및 수단 선택: 안전 자산 중심의 예금/적금, CMA나 MMF처럼 필요 시 입출금이 용이한 상품을 비교해 보세요. 긴급자금은 별도 계좌에 보관하고, 이체 금액은 그 범위 내에서만 조정합니다.
- 진행 점검: 매달 말에 목표 달성 여부를 확인하고 필요하면 이체 금액이나 목표를 조정합니다. 비상금은 3~6개월 생활비 수준으로 유지하는 것을 목표로 삼으면 좋습니다.

원하시면 당신의 월수입, 고정지출, 비상자금 현황, 달성하고 싶은 목표 기간을 알려주세요. 개인 상황에 맞춘 구체적인 자동저축 계획을 함께 만들어 드리겠습니다.


In [50]:
prompt = ChatPromptTemplate.from_messages(
    [('system', '당신은 금융 상담사입니다. 모든 질문에 최선을 다해 답변하십시오.'),
     ('placeholder', '{chat_history}'),
     ('human', '{input}')]
)

In [56]:
chat_history = ChatMessageHistory()
chain = (
    prompt
    | model
    | StrOutputParser()
)
chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,
    input_messages_key='input',
    history_messages_key='chat_history'
)

In [60]:
ai_response = chain_with_message_history.invoke(
    {'input':'저축을 늘리기 위해 무엇을 할 수 있나요?'},
    {'configurable':{'session_id':'unused'}}
)
print(ai_response)

다시 한 번요! 저축을 늘리려면 습관과 구조를 함께 바꾸는 것이 핵심입니다. 바로 적용할 수 있는 간단한 로드맵과 체크리스트를 정리해 드릴게요.

핵심 원칙
- 자동화 우선: 수입이 들어오자마자 먼저 저축으로 이체되도록 설정합니다.
- 지출 관리: 필요한 것과 불필요한 것을 구분하고, 불필요한 지출을 줄이는 습관을 만듭니다.
- 비상금 우선: 생활비의 3–6개월치를 비상금으로 확보합니다.
- 부채 관리: 금리가 높은 부채가 있으면 먼저 상환하거나 재융자를 검토합니다.
- 차근차근 투자: 비상금이 확보되면 남는 자금을 분산 투자로 서서히 늘립니다. 본인의 위험 수용도에 맞춰 진행합니다.
- 세제 혜택 활용: ISA/IRP 같은 세제 혜택 계좌를 현재 상황에 맞게 활용합니다(한도와 규정은 매년 바뀌므로 확인 필요).

바로 시작할 수 있는 실행 방법
1) 재정 상태 파악
- 월수입, 고정지출(주거비, 공과금, 보험 등), 변동지출(식비, 교통, 여가 등), 현재 저축액, 부채 여부를 한 줄로 정리해 보세요.
- 최근 1–3개월 지출 내역을 분석해 돈이 어디로 가는지 확인합니다.

2) 예산 짜고 지출 축소
- 예산 프레이밍: 50/30/20(필수/원하는 것/저축) 또는 60/20/20 방식 같은 분배를 시도.
- 지출 루프홀 제거: 구독 서비스 정리, 외식 줄이기, 불필요한 지출 카테고리 줄이기.
- 목표 설정: 1년 동안 달성하고 싶은 저축 목표를 구체화합니다.

3) 자동화와 비상금
- 급여일 직후나 매월 초에 자동 이체로 일정 금액을 별도 저축계좌로 보냅니다.
- 비상금을 3–6개월 생활비에 맞춰 차곡차곡 모으는 목표를 세웁니다.

4) 부채 관리
- 금리 높은 부채부터 상환하거나 재융자 가능 여부를 확인합니다.
- 매달 최소상환은 유지하고 여유가 있다면 추가 상환으로 이자 부담을 줄이세요.

5) 저축과 투자 구분
- 비상금이 확보되면 남는 금액을 투자로 전환합니다.
- 초보자는 비용이 낮은 인덱스펀드/ETF 등으로 분산 투자 시작을 고려합니다. 본인의 위

In [61]:
ai_response = chain_with_message_history.invoke(
    {'input':'방금 답해준 내용에 대해 조금 더 간략하게 알려줄 수 있나요?'},
    {'configurable':{'session_id':'unused'}}
)
print(ai_response)

물론입니다. 바로 적용하기 쉽도록 간략하게 요약해 드릴게요.

핵심 원칙
- 자동화 우선: 수입이 들어오자마자 저축으로 이체되도록 설정
- 지출 관리: 필요한 것 vs 불필요한 것을 구분하고 불필요한 지출 줄이기
- 비상금 확보: 생활비의 3–6개월치까지 비상금으로 남겨두기
- 부채 관리: 금리 높은 부채부터 우선 상환
- 차근차근 투자: 비상금이 확보되면 남는 돈으로 분산 투자 시작
- 세제 혜택 활용: ISA/IRP 등 세제혜택 계좌 활용 고려

바로 시작하는 간단 실행 방법
- 현재 수입/지출/저축액 한 줄로 정리
- 예산 분배 간단히: 필요/원하는 것/저축으로 50/30/20 또는 60/20/20
- 자동저축 설정: 급여일 직후에 일정 금액 저축 계좌로 이체
- 비상금 목표: 3–6개월 생활비에 맞춰 모으기
- 부채가 있다면 금리 높은 순으로 상환 우선
- 투자: 초보는 비용이 낮은 인덱스펀드/ETF로 소액부터 시작
- 세제혜택 확인: ISA/IRP 가능 여부 점검

4주 간단 로드맵
- 1주차: 수입/지출 파악, 최근 이용 패턴 분석
- 2주차: 예산과 목표 저축률 결정(예: 10–20%)
- 3주차: 자동저축 계좌 설정, 비상금 목표 확정
- 4주차: 첫 달 실적 점검 및 필요 시 조정

간단 체크리스트
- 수입의 10–20%를 자동저축으로 설정
- 비상금 목표 확인 및 달성 여부 점검
- 월 지출 내역 정기 리뷰 및 구독 정리
- 부채 금리 비교 및 상환 우선순위 정하기
- 3개월 후 재정 상태 점검 및 목표 조정

맞춤형 계획이 필요하면 아래 정보를 알려 주세요
- 월수입과 현재 저축액은 어느 정도인가요?
- 부채가 있다면 금리와 남은 원리금은?
- 비상금 목표(3개월치/6개월치 등)
- 투자 관심도와 위험 수용도
- ISA/IRP 등 세제혜택 계좌 이용 여부

원하시면 숫자 예시를 포함한 바로 실행 가능한 맞춤 계획도 같이 만들어 드릴게요.


In [64]:
trimmer = trim_messages(strategy='last', max_tokens=2, token_counter=len)

In [65]:
chain_with_trimming = (
    RunnablePassthrough.assign(chat_history=itemgetter('chat_history') | trimmer)
    | prompt
    | model
    | StrOutputParser()
)

In [66]:
chain_with_trimmed_history = RunnableWithMessageHistory(
    chain_with_trimming,
    lambda session_id: chat_history,
    input_messages_key='input',
    history_messages_key='chat_history'
)

In [67]:
ai_response = chain_with_trimmed_history.invoke(
    {'input':'저는 5년 내에 집을 사기 위해 어떤 재정 계획을 세워야 하나요?'},
    {'configurable':{'session_id':'finance_session_1'}}
)
print(ai_response)

좋은 목표예요. 5년 안에 집을 마련하려면 목표 금액을 먼저 정하고, 그에 맞춰 비상금 포함한 자금 흐름과 신용·세제 혜택까지 함께 계획하는 것이 핵심입니다. 아래에 바로 적용할 수 있는 가이드와 예시를 드립니다.

1) 먼저 정의할 것
- 목표 주택가: 예를 들어 4억~5억원대인지, 서울 인근으로 잡을지 등 지역과 가격 범위를 정합니다.
- 다운페이먼트 비율: 보통 10–20%를 권장합니다. 정책에 따라 다르니 20%를 기본으로 계산해 두는 것이 안전합니다.
- 5년 간의 총 필요 자금 구성: 다운페인트(예: 20%), 매매에 필요한 각종 비용(취득세, 중개수수료 등), 긴급비상금(일반적으로 3–6개월 생활비) 포함.
- 현재 상황 점검: 월수입, 고정지출, 이미 모인 자금, 부채(이자율, 잔여 원리금) 여부.

2) 현재 상황 파악에서 시작하기
- 비상금: 생활비 3–6개월치 비상자금 확보 여부 확인.
- 부채 관리: 금리 높은 부채가 있다면 우선 상환 전략 수립.
- 신용 상태: 주택담보대출 금리와 한도에 영향이 되므로 신용 관리.

3) 5년 로드맵(간단한 실행 계획)
- 목표 금액과 기간에 맞춘 월 저축 목표 산출
  예시: 목표 주택가 P, 다운페이먼트 D%로 D = P × 20%. 60개월(5년) 동안 필요한 순 저축액이 D가 되도록 매월 일정 금액 자동 이체 설정.
  팁: 투자를 최소화하고 안전자산 중심으로 시작하되, 남는 여유자금은 조금씩 중장기 펀드나 단기 채권형에 분산투자해 자금의 실제 목적 달성률을 높일 수 있습니다.
- 자동저축 계좌/계좌 분리
  집 자금 전용 계좌를 만들고 급여일 직후 자동 이체 설정.
- 주택 관련 제도 활용
  - 주택청약종합저축 등 청약 관련 계좌 활용 여부 점검.
  - 세제 혜택 계좌(해당되는 경우) 확인 후 활용 여부 결정.
- 예산 관리
  - 필요 vs 원치 않는 지출 구분, 구독 서비스 정리, 식비/생활비 효율화.
- 신용 관리
  - 연체 없이 신용점수 관리, 대출 신청 시 기존 부채와의 상환 계획

In [68]:
ai_response = chain_with_trimmed_history.invoke(
    {'input':'만약 당장 6억대 집을 사야하는 상황이라면 어떡하죠?'},
    {'configurable':{'session_id':'finance_session_1'}}
)
print(ai_response)

당장 6억대 집을 “지금 바로” 사야 한다면, 자금 조달 구조와 리스크 관리가 핵심이에요. 아래를 참고해 구체적으로 체크하고 실행해 보시길 권합니다.

1) 기본 자금 구조 먼저 확정하기
- 목표 가격: 6억 원대 가정, 다운페이먼트(DP) 최소 기준은 보통 20% 이상 권장. 20%면 1.2억 원이고, 10% 정도로 낮출 수 있는 경우도 있지만 대출 한도와 금리에 영향이 큽니다.
- 필요한 총 자금 구성:
  - DP: 약 1.2억 원(또는 선호하는 비율에 따라 조정)
  - 대출금액: 대략 4.8억 원(6억 - DP)
  - 취득세/중개수수료 등 초기 비용: 대략 가격의 2~3% 수준으로 예상(약 1,200만~3,000만 원대)
  - 이 밖의 예비비/긴급자금: 주택 관련 수리나 예기치 상황을 고려해 남겨두면 좋습니다.

2) 당장 가능한 대출 옵션 확인하기
- 은행 사전심사/사전승인 받기: 가능 금액, 금리, 상환 기간(보통 15~30년), LTV 한도 확인.
- LTV(대출금액/매매가)와 DTI(총부채상환비율) 제약을 현금 흐름과 맞춰 계산.
- 금리 유형 정하기: 고정금리 일부 비중 유지로 금리 상승 리스크 관리. 5년/10년 고정 등 혼합형도 고려.
- 만약 DP가 부족하면: 가족/친지 차입, 매도인 융자(SELLER FINANCING) 가능 여부, 공동투자 등 대안 모색.

3) 즉시 실행 가능한 실행 로드맵
- 1주 내
  - 목표 DP 금액 재확인(가용 자금 정리).
  - 1~2곳 은행에 대출 사전상담/사전심사를 신청해 가능한 한 빠르게 구제 조건(한도·금리)을 확보.
- 2주 내
  - 자금 조달 조합 확정: 예) 본인 자금 + 은행 대출 + 가족 차입 등. 공식 계약 시 필요한 증빙(소득증빙, 자금출처 증빙 등) 준비.
  - 매매 계약서의 조건(잔금일, 계약금 반환 조건, 금리 변동 대비 조항)을 전문가와 점검.
- 1~2개월 내
  - 실질 계약 및 잔금 이행. 초기 비용(취득세, 중개 수수료 등) 지출 계획 세우기.
  - 월별 주택

In [70]:
def summarize_messages(chain_input):
    stored_messages = chat_history.messages
    if len(stored_messages) == 0:
        return False
    
    summarization_prompt = ChatPromptTemplate.from_messages(
        [('placeholder', '{chat_history}'),
         ('user', '이전 대화를 요약해주세요. 가능한 한 많은 세부 정보를 포함하세요.')]
    )

    summarization_chain = (
        summarization_prompt
        | model
        | StrOutputParser()
    )
    summary_message = summarization_chain.invoke({'chat_history':stored_messages})

    chat_history.clear()
    chat_history.add_message(summary_message)

    return True

In [74]:
chain_with_summarization = (
    RunnablePassthrough.assign(message_summarized=summarize_messages)
    | chain_with_trimmed_history
)
ai_response = chain_with_summarization.invoke(
    {'input':'저에게 어떤 재정적 조언을 해주셨나요?'},
    {'configurable':{'session_id':'finance_session_2'}}
)
print(ai_response)

다음은 지금까지 드린 재정 상담의 핵심 내용 요약입니다.

1) 기본 저축 접근 및 실행 로드맵
- 목표: 저축 습관과 재무 구조를 동시에 개선
- 핵심 원칙
  - 자동화 우선: 수입 들어오자마자 저축으로 이체
  - 지출 관리: 필요/불필요 구분 후 축소
  - 비상금 확보: 생활비의 3~6개월 규모로 마련
  - 부채 관리: 금리 높은 부채부터 상환하거나 재융자 검토
  - 차근차근 투자: 비상금 확보 후 잔액으로 분산투자(리스크 허용도 맞춤)
  - 세제 혜택 활용: ISA/IRP 등 검토
- 실행 방법(요약)
  - 현재 재정 상태 파악: 월수입, 고정지출, 변동지출, 저축액, 부채 여부 확인
  - 예산 짜기: 필요/원하는 것/저축으로 50/30/20 또는 60/20/20 등 시도
  - 자동저축/비상금: 급여일 직후 자동 이체, 비상금 목표 3~6개월
  - 부채 관리: 금리 높은 부채 우선 상환
  - 저축 vs 투자 구분: 우선 비상금 확보 후 남는 자금으로 인덱스펀드/ETF 등 분산투자
  - 세제혜택: 상황에 맞춰 ISA와 IRP 활용 여부 검토
- 4주 로드맵(간단)
  - 1주차: 수입/지출/저축 상태 파악
  - 2주차: 예산안 결정 및 목표 저축률 설정
  - 3주차: 자동저축 설정, 비상금 목표 확정
  - 4주차: 첫 달 실적 점검 및 필요 시 조정
- 체크리스트 요약
  - 수입의 10–20%를 자동저축으로 설정
  - 비상금 달성 여부 매달 확인
  - 월 지출 내역 리뷰 및 구독 관리
  - 부채 금리 비교 및 상환 우선순위 정하기
  - 3개월 후 재정 상태 점검 및 목표 조정
- 맞춤형 도움 요청 포인트
  - 월수입/현재 저축액, 부채 현황, 비상금 목표, 투자 위험 수용도, 세제혜택 이용 여부 등 필요 정보 요청

2) 5년 내 집 마련 계획
- 목표 정의: 주택가/지역, 다운페이먼트 비율(일반적으로 20% 권장), 초기 비용 구성
- 현재 상황 점검 포인트: 비상금 여부, 부채 관리 상태, 신용 상태
- 5년 로드맵